#Library Imports

In [ ]:
from scipy.optimize import fmin, minimize, fmin_slsqp
from scipy import interpolate, stats
import numpy as np
import statistics
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.interpolate import CubicSpline
import csv
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import matplotlib.cm as cm
from matplotlib.colors import Normalize, rgb2hex
from matplotlib.colorbar import ColorbarBase

# Functions & Class Definition

##Cost functions

In [ ]:
#Mean squared error cost function, as used by Oerlemans. Exaggerates outliers
def msecost(tc, g1, g2):
    t1, t2, c1 = tc
    m1 = np.zeros(g1.lengths.shape)
    for i in range(g1.lengths.shape[0]):
        m1[i] = ((t1*g1.dldt[i])+(g1.lengths[i]))/c1
    m2 = np.zeros(g2.lengths.shape)
    for i in range(g2.lengths.shape[0]):
        m2[i] = ((t2*g2.dldt[i])+(g2.lengths[i]))/g2.c
    cost = np.sqrt((1/len(m2)*np.sum(np.square((np.subtract(m1, m2))))))
    return cost

#MSE, but with normalised ela for both glaciers, removing dependency on c. This is the one
#to use when looping through all glaciers since it doesnt require estimate of response time
#of one of them
def msecost2(tc, g1, g2):
    t1, t2 = tc
    m1 = np.zeros(g1.lengths.shape)
    for i in range(g1.lengths.shape[0]):
        m1[i] = ((t1*g1.dldt[i])+(g1.lengths[i]))/g1.c
    m1norm = normalize(m1)
    m2 = np.zeros(g2.lengths.shape)
    for i in range(g2.lengths.shape[0]):
        m2[i] = ((t2*g2.dldt[i])+(g2.lengths[i]))/g2.c
    m2norm = normalize(m2)
    cost = np.sqrt(1/len(m2norm)*np.sum(np.square((np.subtract(m1norm, m2norm)))))
    return cost

#Mean absolute error cost function; outliers have less influence
def maecost(tc, g1, g2):
    t1, t2, c1 = tc
    ela1 = np.zeros(g1.lengths.shape)
    for i in range(g1.lengths.shape[0]):
        m = (t1*g1.dldt[i])+(g1.lengths[i])
        ela1[i] = m/c1
    ela1norm = ela1/max(ela1)
    ela2 = np.zeros(g2.lengths.shape)
    for i in range(g2.lengths.shape[0]):
        m = (t2*g2.dldt[i])+(g2.lengths[i])
        ela2[i] = m/g2.c
    ela2norm = ela2/max(ela2)
    cost = (1/g2.lengths.shape[0]*np.sum(np.abs((np.subtract(ela1, ela2)))))
    return cost

#Mean absolute error cost function normalised; outliers have less influence
def maecost2(tc, g1, g2):
    t1, t2 = tc
    ela1 = np.zeros(g1.lengths.shape)
    for i in range(g1.lengths.shape[0]):
        m = (t1*g1.dldt[i])+(g1.lengths[i])
        ela1[i] = m/g1.c
    ela1norm = normalize(ela1)
    ela2 = np.zeros(g2.lengths.shape)
    for i in range(g2.lengths.shape[0]):
        m = (t2*g2.dldt[i])+(g2.lengths[i])
        ela2[i] = m/g2.c
    ela2norm = normalize(ela2)
    cost = (1/g2.lengths.shape[0]*np.sum(np.abs((np.subtract(ela1norm, ela2norm)))))
    return cost

#Derivative cost 1
def dercost1(tc):
    t1, t2 = tc
    ela1 = np.zeros(g1.lengths.shape)
    for i in range(g1.lengths.shape[0]):
        m = (t1*g1.dldt[i])+(g1.lengths[i])
        ela1[i] = m/g1.c
    ela1norm = ela1/max(ela1)
    dela1 = np.gradient(ela1norm)
    ela2 = np.zeros(g2.lengths.shape)
    for i in range(g2.lengths.shape[0]):
        m = (t2*g2.dldt[i])+(g2.lengths[i])
        ela2[i] = m/g2.c
    ela2norm = ela2/max(ela2)
    dela2 = np.gradient(ela2norm)
    cost = 1/palue.lengths.shape[0]*np.sum(np.abs((np.subtract(dela1, dela2))))
    return cost

#Derivative cost 2
def dercost2(tc, g10, g20):
    t1, t2 = tc
    ela1 = np.zeros(g10.lengths.shape)
    for i in range(g10.lengths.shape[0]):
        m = (t1*g10.dldt[i])+(g10.lengths[i])
        ela1[i] = m/g10.c
    ela1norm = ela1/max(ela1)
    dela1 = np.gradient(ela1norm)
    ela2 = np.zeros(g20.lengths.shape)
    for i in range(g20.lengths.shape[0]):
        m = (t2*g20.dldt[i])+(g20.lengths[i])
        ela2[i] = m/g20.c
    ela2norm = ela2/max(ela2)
    dela2 = np.gradient(ela2norm)
    costlist = []
    for i in range(len(ela1norm)):
        #costi = np.abs((dela1[i]-dela2[i])/(dela1[i]+dela2[i]))
        costi = np.abs((dela1[i]-dela2[i])/dela1[i])
        costlist.append(costi)
    cost = sum(costlist)
    return cost

#normalized derivative mse cost
def dercost3(tc, g10, g20):
    t1, t2 = tc
    ela1 = np.zeros(g10.lengths.shape)
    for i in range(g10.lengths.shape[0]):
        m = (t1*g10.dldt[i])+(g10.lengths[i])
        ela1[i] = m/g10.c
    ela1norm = normalize(ela1)
    dela1 = np.gradient(ela1norm)
    ela2 = np.zeros(g20.lengths.shape)
    for i in range(g20.lengths.shape[0]):
        m = (t2*g20.dldt[i])+(g20.lengths[i])
        ela2[i] = m/g20.c
    ela2norm = normalize(ela2)
    dela2 = np.gradient(ela2norm)
    costlist = []
    for i in range(len(dela1)):
        costlist.append(np.square((dela1[i]-dela2[i])/(dela2[i]))/len(dela1))
    cost = np.sqrt(sum(costlist))
    return cost

#normalized derivative mse cost
def dercost4(tc, g10, g20):
    t1, t2 = tc
    ela1 = np.zeros(g10.lengths.shape)
    for i in range(g10.lengths.shape[0]):
        m = (t1*g10.dldt[i])+(g10.lengths[i])
        ela1[i] = m/g10.c
    ela1norm = normalize(ela1)
    dela1 = np.gradient(ela1norm)
    ela2 = np.zeros(g20.lengths.shape)
    for i in range(g20.lengths.shape[0]):
        m = (t2*g20.dldt[i])+(g20.lengths[i])
        ela2[i] = m/g20.c
    ela2norm = normalize(ela2)
    dela2 = np.gradient(ela2norm)
    cost = np.sqrt(np.sum(np.square(np.subtract(dela1, dela2)))/len(dela1))
    return cost

#correlation coefficient cost (r)
def rcost(tc, g10, g20):
    t1, t2 = tc
    m1 = np.zeros(g1.lengths.shape)
    for i in range(g1.lengths.shape[0]):
        m1[i] = ((t1*g1.dldt[i])+(g1.lengths[i]))/g1.c
    m1norm = normalize(m1)
    m2 = np.zeros(g2.lengths.shape)
    for i in range(g2.lengths.shape[0]):
        m2[i] = ((t2*g2.dldt[i])+(g2.lengths[i]))/g2.c
    m2norm = normalize(m2)
    cost = 1-np.corrcoef(m1norm, m2norm)[0,1]
    return cost

#r2 cost, with differences between ela's as dataset and regression as y = 0
def r2cost(tc):
    t1, t2, c1 = tc
    ela1 = np.zeros(morteratsch.lengths.shape)
    for i in range(morteratsch.lengths.shape[0]):
        m = (t1*morteratsch.dldt[i])+(morteratsch.lengths[i])
        ela1[i] = m/c1
    ela2 = np.zeros(palue.lengths.shape)
    for i in range(palue.lengths.shape[0]):
        m = (t2*palue.dldt[i])+(palue.lengths[i])
        ela2[i] = m/g2.c
    diff = np.subtract(ela1, ela2)
    mean = np.mean(diff)
    regr = np.zeros(ela1.shape)
    sstotarray = np.zeros(ela1.shape)
    ssres = np.sum(np.square(diff))
    for i in range(len(sstotarray)):
        sstotarray[i] = diff[i]-mean
    sstot = np.sum(np.square(sstotarray))
    r2 = 1-ssres/sstot
    cost = 1-r2
    return cost, diff

#Huber cost; behaves like mae for small h, mse for large h
def hcost(tc):
    t1, t2, c1 = tc
    ela1 = np.zeros(g1.lengths.shape)
    for i in range(g1.lengths.shape[0]):
        m = (t1*g1.dldt[i])+(g1.lengths[i])
        ela1[i] = m/c1
    ela2 = np.zeros(g2.lengths.shape)
    for i in range(g2.lengths.shape[0]):
        m = (t2*g2.dldt[i])+(g2.lengths[i])
        ela2[i] = m/g2.c
    diff = np.subtract(ela1, ela2)
    h = 1
    costarray = np.zeros(ela1.shape)
    for i in range(costarray.shape[0]):
        if abs(diff[i]) < h:
            costarray[i]=0.5*diff[i]**2
        else:
            costarray[i]=h*np.abs(diff[i])-0.5*h**2
    cost = np.sum(costarray)
    return cost

#Mean absolute error cost function; outliers have less influence
def lccost(tc, g1, g2):
    t1, t2, q = tc
    ela1 = np.zeros(g1.lengths.shape)
    for i in range(g1.lengths.shape[0]):
        m = (t1*g1.dldt[i])+(g1.lengths[i])
        ela1[i] = m/c1
    ela1norm = ela1/max(ela1)
    dela1 = np.gradient(ela1norm)
    ela2 = np.zeros(g2.lengths.shape)
    for i in range(g2.lengths.shape[0]):
        m = (t2*g2.dldt[i])+(g2.lengths[i])
        ela2[i] = m/g2.c
    ela2norm = ela2/max(ela2)
    cost = 1/g2.lengths.shape[0]*np.sum(np.log(np.cosh(((np.subtract(ela1norm, ela2norm))))))
    return cost

##Minimization functions

In [ ]:
#Minimizing functions
#For functions which have c as argument
def minimizer(g10, g20, t10, t20, c10, costfun):
    tc = (t10, t20, c10)
    g1 = g10
    g2 = g20
    tau = fmin(costfun, tc, args=(g1, g2), disp=False) # bounds=[(0, 150),(0, 150)]) #disp=False)
    return tau

#For normalized functions
def minimizer2(g1, g2, t10, t20, costfun):
    tc = (t10, t20)
##    tau = fmin(costfun, tc, args=(g1, g2), disp=False)
    tau = minimize(costfun, tc, args=(g1,g2), bounds=[(0, 200),(0, 200)])
    return tau.x

##Miscellaneous

In [ ]:
##function that opens csv and converts it to array with end year and cumulative lengths
def llister(filename):
    data = np.genfromtxt(filename + '.csv', delimiter=',')[:,:4]
    lengths = data[:,3]
    years = data[:,1]
    data = np.array([years, lengths])
    return data

##function that calculated ela of glacier given a response time
def ela(glacier, t):
    elist = np.zeros(glacier.lengths.shape)
    for i in range(glacier.lengths.shape[0]):
        e = t*glacier.dldt[i]/glacier.c+glacier.lengths[i]/glacier.c
        elist[i] = e
    return elist

#stineman interpolation (monotonic cubic spline). Uses list of years
def pchip(y, l, y0, ymax):
    interp = interpolate.PchipInterpolator(y, l)
    der = interpolate.PchipInterpolator.derivative(interp)
    ynew = np.arange(y0, ymax, 0.5)
    lnew = interp(ynew)
    dldt = der(ynew)
    return ynew, lnew, dldt

#smoothing filter applied onto interpolant
def savgol(l, window, n):
    l = savgol_filter(l, window, n)
    return l

#polynomial interpolation to compare coefficients
def poly(x, y, n):
    coef = np.polyfit(x, y, n)
    yint = np.zeros(len(x))
    for i in range(len(x)):
        for k in range(len(coef)):
            yint[k] = coef[k]*x[i]**(n-k)
    return coef, yint

#Normalize array
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm

#function to select a glacler from the 'glaciers' list using its name
def findglac(glacier):
    return glaciers[names.index(str(glacier))]

#function for checking if data are strictly ascending in time
def ascend(glac):
    notascend = 0
    for k in range(1,len(glac.years)):
        if glac.years[k] <= glac.years[k-1]:
            notascend = 1
    return notascend

#Function to filter length data
#ynew, lnew is the stineman interpolant. filter is then applied on lnew, and gradient is
#found at each point (dldt) and filtered again
def filterer(glacier, window, n, y0, ymax):
    ynew = pchip(glacier.years, glacier.lengths, y0, ymax)[0]
    lnew = pchip(glacier.years, glacier.lengths, y0, ymax)[1]
    years = ynew
    lengths = savgol(lnew, window, n)
    dldt = savgol(np.gradient(lengths, years), window, n)
    return ynew, lengths, dldt

def filterer2(glacier, n):
##    ynew = np.convolve(x, np.ones((N,))/N, mode='same')
    lnew = np.convolve(glacier.lengths, np.ones((n,))/n, mode='same')[0:-1]
    years = glacier.years[0:-1]
    print(lnew, years)
##    lengths = savgol(lnew, window, n)
    dldt = np.gradient(lnew, years)
    return years, lnew, dldt



##Class

In [ ]:
#glacier definition with arrays for end years, lengths, rate of change of length, cm, ela, t)
class glacier:
    def __init__(self, name, years, lengths, dldt, c, ela, t, x, y):
        self.name = name
        self.years = years
        self.lengths = lengths
        self.dldt = dldt
        self.c = c
        self.ela = ela
        self.t = t
        self.x = x
        self.y = y

#Data Imports

In [ ]:
#Open csv file
with open('GLAMOS_Processed.csv', 'r') as f:
    reader = csv.reader(f)
    file = list(reader)

with open('Coords_Processed.csv', 'r') as f:
    reader = csv.reader(f)
    file2 = list(reader)

#Make a list of glacier names (useful for finding glaciers later)
names = ['Silvrettagletscher']
for row in file:
    if str(row[0]) != names[-1]:
        names.append(str(row[0]))

#Make a list of glaciers (each element is an object of the glacier class)
glaciers = []
for i in range(len(names)):
    glaciers.append(glacier(names[i], 0, 0, 0, 0, 0, 0, 0, 0))

#Adding years data to each glacier
for i in range(len(names)):
    ylist = []
    for k in range(len(file)):
        if file[k][0] == names[i]:
            ylist.append(float(file[k][1]))
    glaciers[i].years = np.array(ylist)
    ylist = None

#Adding length data to each glacier
for i in range(len(names)):
    dllist = []
    for k in range(len(file)):
        if file[k][0] == names[i]:
            dllist.append(float(file[k][-1]))
    llist = [dllist[0]]
    for k in range(1,len(dllist)):
        llist.append(llist[k-1]+dllist[k])
    glaciers[i].lengths = np.array(llist)
    dllist = None
    llist = None

#Adding coordinates to each glacier
for i in range(len(names)):
    for k in range(len(file2)):
        if file2[k][0] == names[i]:
            glaciers[i].x = float(file2[k][2][1:])
            glaciers[i].y = float(file2[k][3])

#Removing 'small' data sets (less than 60 years)
small = []

for glac in glaciers:
    if len(glac.years)<60:
        small.append(glac)

for i in small:
    glaciers.remove(i)
    names.remove(i.name)

#Checking if data are strictly ascending in time
notascendlist = []
for glac in glaciers:
    if ascend(glac) != 0:
        notascendlist.append(glac)
for glac in notascendlist:
    glaciers.remove(glac)
    names.remove(glac.name)

#Checking gaps in time series
gaps = [0]
for glac in glaciers:
    for i in range(0,len(glac.years)-1):
        if abs(glac.years[i]-glac.years[i+1]) > 21:
            if gaps[-1] != glac:
                gaps.append(glac)

gaps.remove(0)
for i in gaps:
    glaciers.remove(i)
    names.remove(i.name)

# Glaciers from original data
morteratsch     = glacier('morteratsch', llister('morteratsch')[0], llister('morteratsch')[1], 0, 0, 0, 0, 0, 0)
palue           = glacier('palue', llister('palue')[0], llister('palue')[1], 0, -9, 0, 0, 0, 0)

#Testing Specific Pairs

In [ ]:
#Parameters of filter
n=3
k=41
#Choose glaciers
g2, g1 = findglac('Vadret da Tschierva'), findglac('Vadret da Morteratsch')
##g3, g4 = morteratsch, palue
##g1, g2 = morteratsch, palue
#Set sensitivities
g1.c, g2.c = -1, -1
##g3.c, g4.c = -1, -1
#Find latest and earliest common years of both timesets
ymax = min(max(g1.years), max(g2.years))
##ymax = 2010
ymin = max(min(g1.years), min(g2.years))
#Filter time series using parameters just calculated
g2.years, g2.lengths, g2.dldt = filterer(g2,k,n,ymin,ymax)
g1.years, g1.lengths, g1.dldt = filterer(g1,k,n,ymin,ymax)
##g2.years, g2.lengths, g2.dldt = filterer2(g2,n)
##g1.years, g1.lengths, g1.dldt = filterer2(g1,n)
##g1.lengths = g1.lengths - max(g1.lengths)

##g3.years, g3.lengths, g3.dldt = filterer(g3,61,3,ymin,ymax)
##g4.years, g4.lengths, g4.dldt = filterer(g4,61,3,ymin,ymax)
##g3.lengths = g3.lengths - max(g3.lengths)

plt.plot(g1.years, g1.lengths, label=g1.name)
plt.plot(g2.years, g2.lengths, label=g2.name)
##plt.plot(g3.years, g3.lengths, label=g3.name)
##plt.plot(g4.years, g4.lengths, label=g4.name)
plt.legend(loc='upper right')
plt.show()
##g1.t, g2.t, g1.c = minimizer(g1, g2, 15, 15, -10, msecost)

#Minimization
g1.t, g2.t = minimizer2(g1, g2, 15, 15, msecost2)
##g3.t, g4.t = minimizer2(g3, g4, 50, 50, msecost2)

#Calculate ela, normalized
g1.ela = ela(g1, g1.t)
g2.ela = ela(g2, g2.t)
g2norm = normalize(g2.ela)
g1norm = normalize(g1.ela)

##g3.ela = ela(g3, g3.t)
##g4.ela = ela(g4, g4.t)
##g4norm = normalize(g4.ela)
##g3norm = normalize(g3.ela)

plt.plot(g1.years, g1norm, label = g1.name + ', t = ' + str(round(g1.t, 2)))
plt.plot(g2.years, g2norm, label = g2.name + ', t = ' + str(round(g2.t, 2)))
plt.legend(loc='upper right')
plt.show()
##plt.plot(g3.years, g3norm, label = g3.name + ', t = ' + str(round(g3.t, 2)))
##plt.plot(g4.years, g4norm, label = g4.name + ', t = ' + str(round(g4.t, 2)))
##plt.legend(loc='upper right')
##plt.show()
##plt.plot(g1.years, g1.lengths)
##plt.show()

#Making Distribution

In [ ]:
n = 3
k = 61
#Choose glacier for which distribution is made
gname = 'Vadret da Morteratsch'
#List of response times from each pair
tlist = []
#List of distances from chosen glacier (gname)
dlist = []
#List of names of pairs which work, used later for graph
names2 = []
for i in range(len(glaciers)):
    g1 = findglac(names[i])
    g2 = findglac(gname)

    #Calculate ymax and ymin
    ymax = min(max(g1.years), max(g2.years))
    ymax = 2010
    ymin = max(min(g1.years), min(g2.years))
    ymin = 1960
    #Apply interpolation, filter to smoothen data
    g2years, g2lengths, g2dldt = filterer(g2,k,n,ymin,ymax)
    g1years, g1lengths, g1dldt = filterer(g1,k,n,ymin,ymax)
    #Set sensitivity
    g2c = -1
    #Make new glacier objects just for calculation so that original ones are not changed
    g1new = glacier(g1.name, g1years, g1lengths, g1dldt, -1, 0, 0, g1.x, g1.y)
    g2new = glacier(g2.name, g2years, g2lengths, g2dldt, g2c, 0, 0, g2.x, g2.y)
    ##plt.plot(g1.years, g1.lengths, label=g1.name)
    ##plt.plot(g1.years, g2.lengths, label=g2.name)
    ##plt.legend(loc='upper right')
    ##plt.show()
    #Calculate response times
    g1new.t, g2new.t = minimizer2(g1new, g2new, 15, 15, rcost)
    tlist.append(g2new.t)
    dlist.append(sqrt((g1.x-g2.x)**2+(g1.y-g2.y)**2))
    #Calculate normalized ela for each with this response time
    g1new.ela = ela(g1new, g1new.t)
    g2new.ela = ela(g2new, g2new.t)
    g2norm = normalize(g2new.ela)
    g1norm = normalize(g1new.ela)
##    if names[i] == 'Vadret da Morteratsch':
##        plt.plot(g1new.years, g1norm, label = g1new.name + ', t = ' + str(round(g1new.t, 2)))
##        plt.plot(g2new.years, g2norm, label = g2new.name + ', t = ' + str(round(g2new.t, 2)))
##        plt.legend(loc='upper right')
##        plt.show()
##    if g2new.t < 0:
##        plt.plot(g1new.years, g1new.lengths, label = g1new.name)
##        plt.plot(g2new.years, g2new.lengths, label = g2new.name)
##        plt.legend(loc='upper right')
##        plt.show()
    
##    plt.subplot(7,10,i+1)
##    plt.plot(g1new.years, g1norm)
##    plt.plot(g2new.years, g2norm)
##    plt.yticks([])
##    plt.xticks([])

    names2.append(g1new.name)
    #Delete all variables before next iteration
    g1new = None
    g2new = None
    cost = None
    g1years, g1lengths, g1dldt = None, None, None
    g2years, g2lengths, g2dldt = None, None, None
##plt.show()

#Normalized array of distances from chosen glacier to others
darray=np.zeros(len(dlist))
##for i in range(len(dlist)):
##    if dlist[i]>1000000:
##        dlist[i]=0
dmax = max([i for i in dlist if i<1000000])

for i in range(len(dlist)):
    darray[i]=dlist[i]/dmax

norm = Normalize(vmin=0, vmax=1)
mapper = ScalarMappable(norm, cmap='jet')

names2 = np.array(names2)
tarray = np.array(tlist)
for i in range(len(tarray)-5):
    if abs(tarray[i])>300:
        tarray = np.delete(tarray, i)
        names2 = np.delete(names2, i)
##plt.hist(tarray, bins = 50, density=True, histtype = 'step')
##plt.xlabel('Response Time')
##plt.ylabel('Frequency')
##plt.title('Response time distribution: ' + str(gname))
##plt.show()
##print(np.mean(tarray))

fig = plt.figure(figsize=(13, 8))
ax = fig.add_axes([0.2, 0.1, 0.6, 0.8])
##plt.yticks([])
plt.xlabel('Response time (years)')
plt.ylabel('Glacier couple number')
cbaxes1=fig.add_axes([0.9, 0.1, 0.02, 0.8])

for i in range(len(tarray)):
    if darray[i]>1:
        ax.scatter(tarray[i], i, c='#d3d3d3')
    else:
        ax.scatter(tarray[i], i, c=rgb2hex(mapper.to_rgba(darray[i])))
##cb = ColorbarBase(cbaxes1,cmap='jet',norm=norm,orientation='vertical',extend="both")
##plt.scatter(range(len(tarray)), tarray)
##for i in range(len(tarray)):
##    plt.text(i, tarray[i], names2[i])


cb = ColorbarBase(cbaxes1,cmap='jet',norm=norm,orientation='vertical',extend="both")
plt.show()    
##plt.savefig('test.png', dpi=75)
##plt.close()